In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [3]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    return (lambda_ / 2) * w @ w.T + np.sum(np.vstack((np.zeros(y.shape), (1 - y * (X @ w)))).max(axis=0))

In [4]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    y_train = X @ w
    y_train[y_train > 0] = 1
    y_train[y_train < 0] = -1
    return np.sum(y_train == y) / y.shape[0]

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [5]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    hinge = max(0, (1 - y_n * (x_n @ w)))
    if hinge == 0:
        return num_examples * lambda_ * w
    else:
        return num_examples * (-y_n) * x_n  + lambda_ * w

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [6]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        if it % 10000 == 0:
            print(grad)
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

[ 6.635050e+05  3.304600e+05  5.253700e+05  2.052950e+05  7.555000e+03
  8.210100e+05  3.670000e+03  1.141500e+04  2.347000e+04  1.074925e+06
  6.230000e+03 -7.025000e+03  5.000000e+00  1.937250e+05 -4.540000e+03
  1.404500e+04  2.413600e+05  4.490000e+03  7.060000e+03  1.331500e+05
 -4.665000e+03  1.747550e+06  1.000000e+04  3.254300e+05 -1.935000e+03
 -1.280000e+04  3.144050e+05 -9.485000e+03 -2.400000e+02  6.398350e+05]
iteration=0, cost=3567186280319.23
[ 2.29239861e+04 -2.22985220e+04 -1.29858742e+04  1.74786109e+04
 -1.68714162e+04 -1.18044282e+04 -1.69143430e+04 -3.38937267e+02
  3.41080319e+02  1.86998951e+03 -6.38403055e+02  7.14279187e+02
 -1.65500567e+04  8.13461542e+03 -8.01270108e+00  5.51122617e+01
 -5.39536309e+03  3.02620141e+02  5.23701304e+01  7.75025505e+03
 -1.15658020e+02 -1.67158473e+04 -5.72761007e+02  3.09412773e+04
  2.23406103e+04  2.15070279e+04 -2.28017614e+04 -1.64734587e+04
 -1.67492284e+04 -8.69253065e+02]
iteration=10000, cost=3420872832.6255593
[ 4.3948

## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [8]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    hinge = 1 - y_n * x_n @ w
    new_alpha = old_alpha_n + (lambda_ / (x_n @ x_n))*(1 - y_n * (x_n @ w))
    if new_alpha < 0:
        new_alpha = 0
    elif new_alpha > 1:
        new_alpha = 1
    #Compute optimal new alpha
    #if old_alpha_n == 0:
    #    new_alpha = 0
    #elif hinge > 0:
    #    new_alpha = 1
    #else:
    #    new_alpha = 0.5
    #alpha[n] = new_alpha
    #Compute new w
    w += (1 / lambda_) * x_n * y_n * new_alpha
    return w, alpha

In [13]:
M = np.arange(16).reshape(4,4)
display(M)
m_n = M[2]
m_n.T @ m_n

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

366

In [52]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    y_diag = np.diag(y)
    Q = y_diag @ X @ X.T @ y_diag
    return np.sum(alpha) - (1 / (2 * lambda_)) * alpha.T @ Q @ alpha

In [54]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 10000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
        if it % 10 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print(primal_value, dual_value)
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

698402922674.1354 -356843916.43185
iteration=0, primal:698402922674.13538, dual:-356843916.43185, gap:698759766590.56726
211594843556.60797 -195583591.46979952
iteration=10, primal:211594843556.60797, dual:-195583591.46980, gap:211790427148.07776
97473037057.7303 -53237793.500350095
iteration=20, primal:97473037057.73030, dual:-53237793.50035, gap:97526274851.23065
215270093139.3534 -165348172.4336997
iteration=30, primal:215270093139.35339, dual:-165348172.43370, gap:215435441311.78708
355717663481.4634 -320205341.6599002
iteration=40, primal:355717663481.46338, dual:-320205341.65990, gap:356037868823.12329
995497349856.6698 -496144312.97925097
iteration=50, primal:995497349856.66980, dual:-496144312.97925, gap:995993494169.64905
619649781971.2169 -951877856.2197013
iteration=60, primal:619649781971.21692, dual:-951877856.21970, gap:620601659827.43665
513592169240.9718 -1463623750.03245
iteration=70, primal:513592169240.97180, dual:-1463623750.03245, gap:515055792991.00427
51359216924

KeyboardInterrupt: 